In [3]:
import numpy as np
import pandas as pd
from Preprocessing.evaluation_function import search_moving_average


In [13]:
datapath = r"K:\EWB_010_Abteilung\100_Mitarbeiter\Bender\0 GIT\BLDC_Check_Data\MW2\002_026_V004_R065_MW_60_Pyro_G_TOA24_006.csv"
df = pd.read_csv(datapath,delimiter=';', skiprows=3, decimal=',')

In [7]:
#datapath = r\"K:/EWB_070_Steam_Tech/011_measurements/Coco_Read_In\MeasITtest_0.csv"
datapath = r"K:\EWB_070_Steam_Tech\011_measurements\Coco_Read_In\MeasITtest_0.csv"

df = pd.read_csv(datapath,delimiter=';',  on_bad_lines='skip') #delimiter='\t',

In [10]:
df = pd.read_csv(datapath,delimiter=';',  on_bad_lines='skip', skiprows=range(4, 6), decimal=',',na_values='#NV')

In [16]:
print(df)

       Name  CPM1_CoolFanSpeed  CPM1_ConvectionFanSpeed  \
0      max.                 20                       22   
1         0                  0                       22   
2         1                  0                       22   
3         2                  0                       22   
4         3                  0                       22   
...     ...                ...                      ...   
6131   6130                 20                       22   
6132   6131                 20                       22   
6133   6132                 20                       22   
6134   6133                 20                       22   
6135   6134                 20                       22   

      CPM1_ConvFanDirection  CPM1_ConvectionFanType  CPM1_RelaisState1  \
0                         1                       4                  7   
1                         0                       4                  5   
2                         0                       4                  

In [21]:
# Set the maximum column width
pd.set_option('display.max_colwidth', None)

In [10]:
df.head

<bound method NDFrame.head of   Instrument:               Unnamed: 1
0    created:  09:43:20,854 27.08.2024>

In [14]:
from Preprocessing.evaluation_function import search_moving_average, check_saddle_point_occurred
from Preprocessing.Filter_Routines import preprocess_with_period_filter2
from Preprocessing.preprocessing_function import cut_BLDC_data_b,cut_BLDC_data,resample_BLDC_data, cStaticVariables,Filter_Accumulated_Average, temperature_comp, get_nom_speed



motor_sample_rate=1
temperature_compensation=False
min_samples=500 // motor_sample_rate # "minimum time" start of evaluation
stop_signal=2 # amount of samples short > long
samples_short=30*5 // motor_sample_rate#samples for short moving average
samples_long=85*5 // motor_sample_rate #samples for long average
acceptance_limit = {
    "power_min": 2500, #minimum value of power consumption
    "speed_low": 10, #rpm_nominal - value --> upper bound
    "speed_high": 11640, #rpm
    "temp_min":20, #Temp_Set_Point - value ---> lower bound
        }
lower_cutoff_threshold=0
filter_period_1 = 350
lower_cutoff_threshold=0
start_end_padding_enabled=False
padding_value=0
group_delay_compensation_enabled=False
group_delay_compensation='default'

# Assuming 'df' is your DataFrame
print(len(df))
df = df.iloc[:-5000]

df.fillna(-1, inplace=True)
# Offset Correction with Temp set point
Temp_Minimum_Value=max(df['CPM1_TempSetPoint'])-acceptance_limit['temp_min']

Clean_Power_Signal=Filter_Accumulated_Average(df,Temp_Minimum_Value,acceptance_limit,temperature_compensation)
df["cps"]=Clean_Power_Signal
Clean_Power_Signal_C= np.trim_zeros(Clean_Power_Signal, 'f') #remove leading zeros

if len(Clean_Power_Signal_C) == 0: #Check for sufficient data
    print(': no power data or power data out of bounds')
    raise ValueError('no power data or power data out of bounds')
    

#filter?
filter_period = filter_period_1
y = np.array(Clean_Power_Signal_C)
Clean_Power_Signal_Smooth = preprocess_with_period_filter2(y,lower_cutoff_threshold,filter_period,start_end_padding_enabled,padding_value,group_delay_compensation_enabled)

#ma method
saddle_occurred, end_sample_ma = check_saddle_point_occurred(Clean_Power_Signal_Smooth, min_samples, stop_signal, samples_short, samples_long)


6136


In [23]:
print(saddle_occurred)

False


In [15]:
import matplotlib.pyplot as plt

Clean_Power_Index=df.index[len(df.index)-len(Clean_Power_Signal_Smooth):]
endsample_corrected=Clean_Power_Index[end_sample_ma] #add cut time
endtime_corrected_minute=round(endsample_corrected/60,2) #calculate endtime in minutes
print(': endsample= ' + str(endsample_corrected) + ' endtime= ' + str(endtime_corrected_minute) + ' min ' + str(end_flag))
#%% ----- PLOT RESULTS ----

fig, (ax1, ax3) = plt.subplots(1, 2, figsize=(16, 8))
p1, = ax1.plot(df['Send_HotAirFan_Power'], color='black', linewidth=0.5,label="original power")
p8, = ax1.plot(Clean_Power_Index,Clean_Power_Signal_Smooth, color='red',label="smoothed power")
ax2 = ax1.twinx()  # Erstelle eine zweite y-Achse
p2, = ax2.plot(Clean_Power_Index,Clean_Power_Signal_Smooth, color='green',label="smoothed power")
#ax1.plot(Clean_Power_Signal, color='blue', linewidth=0.8)
#    ax1.legend(['original power','smoothed power'])
ax1.legend(handles=[p1, p2])
ax1.set_xlabel('time (s)')
ax1.set_ylabel('power (mW)')
#ax2.set_ylabel(power_smooth_label)

ax4 = ax3.twinx()  # Erstelle eine zweite y-Achse
ax5 = ax3.twinx()  # Erstelle eine zweite y-Achse
ax5.spines.right.set_position(("axes", 1.1))

p3, = ax3.plot(Clean_Power_Index,Clean_Power_Signal_Smooth, color='green',label="smoothed power")
p4, = ax3.plot(Clean_Power_Index[end_sample_ma],Clean_Power_Signal_Smooth[end_sample_ma], marker='o', color='black', linewidth=0.5,label="calculated end-point")
p5, = ax4.plot(df['Send_Oxygen'], color='blue', linewidth=0.5,label="oxygen")
p6, = ax5.plot(df['CPM1_TempPT'], color='red', linewidth=0.5,label="temperature")
#p7, = ax3.plot(Clean_Power_Index,Clean_Power_Signal_Smooth2, color='black',label="smoothed power2")

#ax2.legend(['corrected','corrected&cutted','smoothed'])
ax3.legend(handles=[p3, p4,p5,p6],loc='center right')
#    ax3.legend(['smoothed power','difference long-short'], loc='upper left')
#    ax4.legend(['Oxygen'], loc='upper right')
ax3.set_xlabel('samples')
ax3.set_ylabel('power')
ax4.set_ylabel('Oxygen (%)')
ax5.set_ylabel('Temperature (°C)')

plt.text(0.5, 0.93, 'endsample: '+str(endsample_corrected)+'  |  endtime: '+str(endtime_corrected_minute)+' min' +'  |  method: ' + evaluation_name+ '  |  successful: ' + str(end_flag)+'  |  temperature comp: ' + str(temperature_compensation), ha='center', va='center', transform=fig.transFigure)
plt.savefig(folder_plot + filename[:-4] + '_smoothed_Power.png',dpi=400)
plt.show()
plt.close()


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
import numpy as np
import pandas as pd
import tkinter as tk
import threading
from Preprocessing.evaluation_function import search_moving_average, check_saddle_point_occurred
from Preprocessing.Filter_Routines import preprocess_with_period_filter2
from Preprocessing.preprocessing_function import cut_BLDC_data_b,cut_BLDC_data,resample_BLDC_data, cStaticVariables,Filter_Accumulated_Average, temperature_comp, get_nom_speed
import time

class CocoReaderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Coco_reader")
        self.root.geometry("400x300")  # Width x Height
        self.stop = False
        
        # Placeholder for the writer thread
        self.writer_thread = None
        self.reader_thread = None

        
        # Button to start the writer thread
        self.start_button = tk.Button(
            self.root, text="Start", command=self.start_thread
        )
        self.start_button.pack(pady=20)

        # Bind the window's close button to the close_app function
        self.root.protocol("WM_DELETE_WINDOW", self.close_app)

    def dataReader(self, datapath = r"K:\EWB_070_Steam_Tech\011_measurements\Coco_Read_In\MeasITtestlive.csv"):
        motor_sample_rate=1
        temperature_compensation=False
        min_samples=500 // motor_sample_rate # "minimum time" start of evaluation
        stop_signal=2 # amount of samples short > long
        samples_short=30*5 // motor_sample_rate#samples for short moving average
        samples_long=85*5 // motor_sample_rate #samples for long average
        acceptance_limit = {
            "power_min": 2500, #minimum value of power consumption
            "speed_low": 10, #rpm_nominal - value --> upper bound
            "speed_high": 11640, #rpm
            "temp_min":20, #Temp_Set_Point - value ---> lower bound
                }
        lower_cutoff_threshold=0
        filter_period_1 = 350
        lower_cutoff_threshold=0
        start_end_padding_enabled=False
        padding_value=0
        group_delay_compensation_enabled=False
        group_delay_compensation='default'
        time.sleep(2)
        while self.stop!=True:
            try:
                #df = pd.read_csv(datapath,delimiter=';', skiprows=3, decimal=',')
                df = pd.read_csv(datapath)

                df.fillna(-1, inplace=True)
                # Offset Correction with Temp set point
                Temp_Minimum_Value=max(df['CPM1_TempSetPoint'])-acceptance_limit['temp_min']

                Clean_Power_Signal=Filter_Accumulated_Average(df,Temp_Minimum_Value,acceptance_limit,temperature_compensation)
                df["cps"]=Clean_Power_Signal
                Clean_Power_Signal_C= np.trim_zeros(Clean_Power_Signal, 'f') #remove leading zeros

                if len(Clean_Power_Signal_C) == 0: #Check for sufficient data
                    print(': no power data or power data out of bounds')
                    #raise ValueError('no power data or power data out of bounds')   
                else:
                    #filter?
                    filter_period = filter_period_1
                    y = np.array(Clean_Power_Signal_C)
                    Clean_Power_Signal_Smooth = preprocess_with_period_filter2(y,lower_cutoff_threshold,filter_period,start_end_padding_enabled,padding_value,group_delay_compensation_enabled)

                    #ma method
                    saddle_occurred, end_sample_ma = check_saddle_point_occurred(Clean_Power_Signal_Smooth, min_samples, stop_signal, samples_short, samples_long)

                    print(saddle_occurred)

                    if(saddle_occurred==True):
                        print("Sattel Punkt gefunden! Sample: " + end_sample_ma)
                        self.stop=True
                        break
            except Exception:
                print("could not read the file this time")
                print(Exception)

            time.sleep(2) 
        
        return end_sample_ma

    def dataWriter(self, file_path_from = r"K:\EWB_010_Abteilung\100_Mitarbeiter\Bender\0 GIT\BLDC_Check_Data\MW2\002_026_V004_R065_MW_60_Pyro_G_TOA24_006.csv", file_path_to = r"K:\EWB_070_Steam_Tech\011_measurements\Coco_Read_In\MeasITtestlive.csv"):
        df = pd.read_csv(file_path_from,delimiter=';', skiprows=3, decimal=',')
        counter = 1
        while self.stop!=True:
            print("writing data...")
            print(df.iloc[counter])
            df.iloc[:counter].to_csv(file_path_to)
            # if(self.stop==True):
            #     print("Sattel Punkt gefunden! Sample: " + end_sample_ma)
            #     break
            counter += 10
            time.sleep(1) 
        print("Sattel Punkt gefunden! Sample: " + end_sample_ma)
        return end_sample_ma

    def start_thread(self):
        # Only start the writer thread if it's not already running
        if not self.writer_thread or not self.writer_thread.is_alive():
            self.writer_thread = threading.Thread(target=self.dataWriter, daemon=True)
            self.writer_thread.start()
        
        if not self.reader_thread or not self.reader_thread.is_alive():
            # Start the background thread for reading data
            self.reader_thread = threading.Thread(target=self.dataReader, daemon=True)
            self.reader_thread.start()
    
    def close_app(self):
        self.stop=True

# Initialize and run the app
if __name__ == "__main__":
    root = tk.Tk()
    app = CocoReaderApp(root)
    root.mainloop()

writing data...
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out of bounds
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out of bounds
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out of bounds
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out of bounds
writing data...
writing data...
could not read the file this time
<class 'Exception'>
writing data...
: no power data or power data out of bounds
writing data...
writing data...
: no power data or power data out 

writing data...
could not read the file this time
<class 'Exception'>
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
False
writing data...
False
writing data...
writing data...
False
writing data...
False
writing data...
False
writing data...
writing data...
could not read the file this time
<class 'Exception'>
writing data...
False
writing data...
False
writing data...
False
writing data...
writing data...
could not read the file this time
<class 'Exce